In [1]:
!pip install -q transformers datasets accelerate librosa soundfile torchaudio evaluate
!pip install -q jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# Tải toàn bộ dataset
dataset = load_dataset("gilkeyio/librispeech-alignments")


Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

train_clean_360-00036-of-00049.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

...

train_clean_360-00039-of-00049.parquet:   0%|          | 0.00/476M [00:00<?, ?B/s]

train_clean_360-00040-of-00049.parquet:   0%|          | 0.00/474M [00:00<?, ?B/s]

train_clean_360-00041-of-00049.parquet:   0%|          | 0.00/478M [00:00<?, ?B/s]

train_clean_360-00042-of-00049.parquet:   0%|          | 0.00/473M [00:00<?, ?B/s]

train_clean_360-00043-of-00049.parquet:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
dataset

In [ ]:
dataset = dataset["train_clean_100"].train_test_split(test_size=0.1)

In [ ]:
dataset

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor
import torchaudio

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="en",
    task="transcribe"
)

In [ ]:
import torchaudio
import torch
import numpy as np

def preprocess_function(batch):
    # Xử lý audio
    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # Xử lý text với timestamps
    formatted_text = " ".join(
        [f"<|{word['start']:.2f}|>{word['word']}"
         for word in batch["words"]]
    )

    batch["labels"] = processor.tokenizer(
        formatted_text,
        max_length=448,  # Giới hạn tối đa theo model
        truncation=True,  # Tự động cắt bỏ phần vượt quá
    ).input_ids

    return batch

In [ ]:
temp_dataset = dataset

In [ ]:
temp_dataset = temp_dataset.map(preprocess_function, remove_columns=temp_dataset['train'].column_names, num_proc=4)

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-timestamp",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",  # Đánh giá sau mỗi epoch
    save_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=10,
    fp16=True,
    logging_steps=100,  # Log mỗi 100 bước
    report_to=["tensorboard"],  # Tắt báo cáo lên MLflow
    load_best_model_at_end=True,  # Tải model tốt nhất cuối cùng
    metric_for_best_model="wer",  # Dùng WER để chọn model
    greater_is_better=False,  # WER càng thấp càng tốt
)

In [ ]:
# Lưu tokenizer và feature extractor vào thư mục output
tokenizer.save_pretrained(training_args.output_dir)
feature_extractor.save_pretrained(training_args.output_dir)

In [ ]:
from transformers import Seq2SeqTrainer


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=temp_dataset["train"],
    eval_dataset=temp_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Thêm hàm tính WER
)

trainer.train()

In [ ]:
from google.colab import files

!zip -r whisper-timestamp.zip ./whisper-timestamp
files.download("whisper-timestamp.zip")

In [ ]:
trainer.push_to_hub()